The purpose of this notebook study is to examine the top 100 languages used globally with respect to native and non native speakers. 

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
print ("Good to go")

In [ ]:
file_pathway = "../input/100-most-spoken-languages-around-the-world/Top 100 Languages.csv"
data = pd.read_csv(file_pathway, decimal= ',')
data.dtypes

In [ ]:
data.head(5)

As of now, our Native Speakers column is producing values with ".0" trailing because this column data type is float. 

We'll have to change this to integer after missing values (NaNs) are removed. 

Additonally languages are indexed 0-99 as opposed to 1-100. 
The following code is used to reflect the necessary change.

In [ ]:
data.index=np.arange(1,len(data)+1)

In [ ]:
data.columns=(["Language","Total_Speakers","Native_Speakers","Origin"])
data.Language=data.Language.astype('category')
data.Origin=data.Origin.astype('category')

There are 42 different Indo-European languages in the database that account for more than half (55.7%) of all the total speakers among the 100 languages when grouped by language origins.

In [ ]:
data['Origin'].value_counts()

In [ ]:
fig = px.pie(data, values='Total_Speakers', names='Origin', title="Percentage Breakdown of Language Groups")
fig.show()

In [ ]:
fig=px.bar(data, x="Language", y="Total_Speakers", color="Origin", title="Total Speakers Grouped by Origin",
          width=1500, height=800)
fig.update_yaxes(ticks="outside", tickwidth=2, tickcolor='crimson', ticklen=10)
fig.update_layout(
    title="Total Speakers of Languages Grouped By Origin",
    title_font_size=20,
    yaxis_title="Total Speakers",
    legend_title="Language Origin",
    font=dict(
        family="Courier New, monospace",
        color="Black"))
fig.show()

Before looking into the data further, missing values will be dealt with. 

In [ ]:
null_columns=data.columns[data.isnull().any()]
data[null_columns].isnull().sum()

In [ ]:
is_NaN=data.isnull()
NaN_rows=is_NaN.any(axis=1)
rows_with_NaN=data[NaN_rows]
print(rows_with_NaN)

There are four languages in the database that have missing values for the number of Native Speakers. 

For these languages only the number of total speakers is provided.
They are:
- Standard Arabic (273.9 million, Afro-Asiatic)
- Filipino (45 million, Austronesian)
- Nigerian Pidgin (30 million, Indo-European) 
- Cameroonian Pidgin (12 million, Indo-European). 

In terms of Total Speakers, these are the 6th, 36th, 50th and 96th most spoken language in the world respectively. 

We will delete these because when comparing Native Speakers they do not add significance.

In [ ]:
data.drop([6,36,50,96], inplace=True)

In addition to converting the Native Speakers column into an integer data type following the elimination of NaNs, we will create 2 columns, Insularity and Non Native. Insularity is expressed as a percentage of the number of Native Speakers divided by Total Speakers for a given language. The higher the outcome, the more insular the language is as speakers are mostly native. Conversely, the lower the number, the more the given language is being spread amongst non native speakers as second or third languages acquired etc.

Non Native is simply calculated as the number of Total Speakers subtracted by the number of Native Speakers. This column tells us the number of speakers who have learned said language perhaps for work, travel, translation, pleasure or have picked it up along the way.

We will round the Insularity to 4 decimal places rather than the standard 2. The reason being, a minor number of Non Native speakers will falsely round to 100% insularity. Here is a demonstration of this inaccuracy: 

In [ ]:
data.Native_Speakers=data.Native_Speakers.astype('int')
data['Non_Native']=data['Total_Speakers'] - data['Native_Speakers']
data.Non_Native=data.Non_Native.astype('int')
data['Insularity']=data['Native_Speakers']/data['Total_Speakers']*100
data['Insularity']=data['Insularity'].round(decimals=2)
total_insularity=data.loc[data['Insularity'] == 100]
total_insularity.head(10)

As you can see the Eastern Punjabi language in the database records 470 Non Native speakers, however Insularity is falsely rounded to 100%. To reflect a more accurate reading of Insularity, 4 decimals places will be required. 

In [ ]:
data['Insularity']=data['Native_Speakers']/data['Total_Speakers']*100
data['Insularity']=data['Insularity'].round(decimals=4)
total_insularity=data.loc[data['Insularity'] == 100]
len(total_insularity)

In [ ]:
#Languages with 100% insularity based on data provided
total_insularity

In [ ]:
fig = px.bar(total_insularity, x="Language", y="Total_Speakers", color='Origin', width=1000, height=700,title="Total Insularity Languages by Origin")
fig.update_xaxes(tickangle=90, tickfont=dict(family='Courier New', color='black', size=14))
fig.update_yaxes(ticks="outside", tickwidth=2, tickcolor='crimson', ticklen=10)
fig.update_layout(
    title="Completely Insular Languages Grouped By Origin",
    title_font_size=20,
    yaxis_title="Total Speakers",
    legend_title="Language Origin",
    font=dict(
        family="Courier New, monospace",
        color="Black"))
fig.show()

There are a total of 36 languages in the database in which Total Speakers is equal to the number of Native Speakers, indicating that there are zero Non Native Speakers. Rather than labelling these languages as unforgivingly insular, a more realistic picture of this may simply be a result of incomplete data. For instance, the Korean language is spoken by several non Koreans which can be observed whenever spending time there. 

On another elaborative note, the language belongs to origin group of Koreanic. Interestingly enough however, both the Korean and the Tamil languages (Dravidian origin) make use of the same word or sound we can say for the words meaning "mother" and "father". This could be a great discussion to be had between Jonathan D. Ripley of the Department of South Asian Studies at Harvard University and researcher Jung Nam Kim that may lead to Chola and Song dynasty unravellings. Of course such talks can only safely occur outside of Ontario Canada. 

Nonetheless for the purpose of this notebook, we will exclude all languages in the database without non native speakers as they lend no further comparative value. 

After dropping these 36 languages in which Non Native information is absent in addition to the 4 languages dropped previously for missing data, we will be left with 60 of the inital 100 in the database to examine.

In [ ]:
data.drop(data.loc[data['Non_Native']==0].index, inplace=True)
len(data)

In [ ]:
most_total=data.sort_values(by='Total_Speakers', ascending=False)
ab=most_total.head(10)
ab

When taking a further look into the Top 10 most spoken languages in terms of Total Speakers by way of Native and Non Native Speakers, a visualization articulates the workings of Insularity on the population on a language's total number of speakers. 
In the top 10 most spoken group, fours languages that stand out in terms of reach outside of Native Speakers are:

- English (33.47% insularity; most spoken language)
- French (27.58% insularity; 5th most spoken language)
- Indonesian (21.82% insularity; 10th most spoken language)
- Urdu (40.32% insularity; 11th most spoken language)

These low levels of insularity amongst these popular languages all indicate that more than 50% (for English two-thirds) of all speakers are Non Natives!

In [ ]:
fig = px.bar(ab, x="Language", y=["Native_Speakers","Non_Native"], title="Breakdown of 10 Most Spoken Languages")
fig.show()

There are a total of 15 languages in the database in which the majority (50%+) of speakers are Non Natives.
More simply, these languages have more Non Native speakers today than Native speakers.
Namely from least insular, they are as follows:

- Swahili (16.3% - insert 2pac lyric here)
- Jula (17.68%)
- Indonesian (21.82%)
- French (27.58)
- Bamanankan (29.09%)
- English (33.47%)
- Northern Sotho (33.73%)
- Thai (34.56%)
- Urdu (40.32%)
- Afrikaans (41.26%)
- Southern Sotho (41.59%)
- Sadri (42.30%)
- Setswana (42.55%)
- Xhosa (42.66%)
- Zula (43.48%)

In [ ]:
least_insular=data.loc[data['Insularity'] < 50]
least_insular.sort_values('Insularity')

In [ ]:
fig = px.sunburst(data, path=['Origin','Language'], values='Total_Speakers',
                  color='Non_Native', color_continuous_scale='RdBu', title="Total and Non Native Speakers")
fig.show()

In [ ]:
fig = px.scatter(data, x="Total_Speakers", y="Native_Speakers", size="Non_Native", color="Origin", 
                 title="With Bubble Size as Non Native Speakers - Total Speakers vs. Native Speakers",
           hover_name="Language", log_x=True, log_y=True,size_max=75,width=900, height=600)
fig.update_layout(showlegend=True,
    xaxis={'title':'Total Number of Speakers',},
    yaxis={'title':'Total Number of Native Speakers'})
fig.show()

In [ ]:
most_insular=data.loc[data['Insularity']>85]
len(most_insular)

On the flip side, by arbitrary methodology if we select greater than 85% to signify that a language is insular, there are 31, or just over half of our reduced from 100 to 60 languages in the database that qualify in being insular. 

Seeing Spanish, Portuguese, Italian, Polish and Japanese as insular is particulary surprising given the use of these languages in Rosetta Stone products. 
Nonetheless, examining the Total Number of Speakers which is commonly done, does not quite reveal the true workings of language reach. 

In [ ]:
most_insular.sort_values('Insularity')